In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

In [2]:
db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "chatbot"

db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")

In [3]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['customer', 'order', 'product', 'shipment']

CREATE TABLE `order` (
	order_id INTEGER NOT NULL, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	status VARCHAR(255), 
	quantity INTEGER, 
	total_price DECIMAL(10, 2), 
	PRIMARY KEY (order_id), 
	CONSTRAINT order_ibfk_1 FOREIGN KEY(customer_id) REFERENCES customer (customer_id), 
	CONSTRAINT order_ibfk_2 FOREIGN KEY(product_id) REFERENCES product (product_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from order table:
order_id	customer_id	product_id	order_date	status	quantity	total_price
300	100	200	2024-06-01	Shipped	1	999.99
301	101	201	2024-06-02	Processing	2	1599.98
302	102	202	2024-06-03	Delivered	1	199.99
*/


CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	first_name VARCHAR(255), 
	last_name VARCHAR(255), 
	email VARCHAR(255), 
	phone_number VARCHAR(255), 
	address VARCHAR(255), 
	PRIMARY KEY (customer_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_090

In [4]:
db.run("SELECT * FROM Customer;")

"[(100, 'John', 'Doe', 'john.doe@example.com', '123-456-7890', '123 Elm St.'), (101, 'Jane', 'Smith', 'jane.smith@example.com', '234-567-8901', '234 Oak St.'), (102, 'Alice', 'Johnson', 'alice.johnson@example.com', '345-678-9012', '345 Pine St.'), (103, 'Bob', 'Williams', 'bob.williams@example.com', '456-789-0123', '456 Maple St.'), (104, 'Carol', 'Brown', 'carol.brown@example.com', '567-890-1234', '567 Cedar St.'), (105, 'David', 'Jones', 'david.jones@example.com', '678-901-2345', '678 Birch St.'), (106, 'Eve', 'Garcia', 'eve.garcia@example.com', '789-012-3456', '789 Walnut St.'), (107, 'Frank', 'Miller', 'frank.miller@example.com', '890-123-4567', '890 Chestnut St.'), (108, 'Grace', 'Davis', 'grace.davis@example.com', '901-234-5678', '901 Redwood St.'), (109, 'Hank', 'Martinez', 'hank.martinez@example.com', '012-345-6789', '123 Sequoia St.')]"

Test the access to the environment variables

In [7]:
from dotenv import load_dotenv
import os

print("Environment variables are loaded:", load_dotenv())

Environment variables are loaded: True


Test GPT

In [10]:
from openai import AzureOpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_API_DEPLOYMENT_NAME"),
    messages=messages
)

In [11]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


SQL query chain

In [8]:
# Load the LLM
from langchain.chat_models import AzureChatOpenAI
import os

model_name = os.getenv("AZURE_OPENAI_API_DEPLOYMENT_NAME")
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

c:\Nikhil\CustomerChatBot\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [17]:
from langchain.chains import create_sql_query_chain

In [18]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({'question': "How may Customer are there"})
print(response)

SELECT COUNT(*) AS `Total Customers` FROM customer


In [19]:
db.run(response)

'[(10,)]'

In [16]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

QuerySQLDataBaseTool (connect this to chain) 

In [20]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [21]:
write_query = create_sql_query_chain(llm,db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({'question': "How may Customer are there"})

'[(10,)]'

Answer the question in a user friendly manner

In [19]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser # more structured and usable string format.
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough # allows you to pass inputs unchanged

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Answer Chain
answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({'question': "How may Customer are there"})

'There are 3 customers.'

Agents

In [20]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [21]:
agent_executor.invoke(
    {
       "input": "Customer who buy laptop?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


customer, order, product, shipment
Invoking: `sql_db_schema` with `{'table_names': 'customer, order, product, shipment'}`



CREATE TABLE `order` (
	order_id INTEGER NOT NULL, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	status VARCHAR(255), 
	quantity INTEGER, 
	total_price DECIMAL(10, 2), 
	PRIMARY KEY (order_id), 
	CONSTRAINT order_ibfk_1 FOREIGN KEY(customer_id) REFERENCES customer (customer_id), 
	CONSTRAINT order_ibfk_2 FOREIGN KEY(product_id) REFERENCES product (product_id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from order table:
order_id	customer_id	product_id	order_date	status	quantity	total_price
1	1	1	2024-06-01	Shipped	1	999.99
2	2	2	2024-06-05	Processing	2	1399.98
3	3	3	2024-06-10	Delivered	1	199.99
*/


CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	first_name VARCHAR(255), 
	last_name VARCHAR(255), 
	email VARCHAR(2

{'input': 'Customer who buy laptop?',
 'output': 'The customer who bought a laptop is John Doe.'}

2nd method

Create a SQL Chain

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [11]:
prompt.format(schema="my schema",question="How many users are there?")

"Human: Based on the table schema below, write a SQL query that would answer the user's question:\nmy schema\n\nQuestion: How many users are there?\nSQL Query:"

In [12]:
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [13]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser # more structured and usable string format.
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough # allows you to pass inputs unchanged

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Answer Chain
answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({'question': "How may Customer are there"})